## Problema KNAPSACK_01
* https://people.sc.fsu.edu/~jburkardt/datasets/knapsack_01/knapsack_01.html

In [1]:
!pip install deap

In [2]:
import random
import numpy
from deap import base
from deap import creator
from deap import algorithms
from deap import tools

In [3]:
espaco_disponivel = 165

In [4]:
capacidade = [23, 31, 29, 44, 53, 38, 63, 85, 89, 82]

In [5]:
custo = [92,57,49,68,60,43,67,84,87,72]

In [6]:
produtos = []
for i in range(len(capacidade)):
  produtos.append(['Item'+str(i),capacidade[i],custo[i]])

In [7]:
produtos

[['Item0', 23, 92],
 ['Item1', 31, 57],
 ['Item2', 29, 49],
 ['Item3', 44, 68],
 ['Item4', 53, 60],
 ['Item5', 38, 43],
 ['Item6', 63, 67],
 ['Item7', 85, 84],
 ['Item8', 89, 87],
 ['Item9', 82, 72]]

In [8]:
# Retorna os valores de custo e peso da solução
def fitness_test(solucao):
  custo = 0
  soma_espaco = 0
  for i in range(len(solucao)):
    if solucao[i] == 1:
      custo += produtos[i][2]
      soma_espaco += produtos[i][1]
  if soma_espaco > espaco_disponivel:
    custo = 0
  return custo, soma_espaco

In [9]:
# Solução ótima
# test= [1,0,0,0,0,0,0,0,0,1]
sol = [1,1,1,1,0,1,0,0,0,0]
fit, peso = fitness_test(sol)
print(f'Fitness: {fit} - Peso: {peso}')

Fitness: 309 - Peso: 165


In [10]:
# Cria a função fitness Oficial
# Retorna apenas o custo no formato DEAP (com vírgula)
def fitness_function(solucao):
  custo = 0
  soma_espaco = 0
  for i in range(len(solucao)):
    if solucao[i] == 1:
      custo += produtos[i][2]
      soma_espaco += produtos[i][1]
  if soma_espaco > espaco_disponivel:
    custo = 0
  return custo,

In [11]:
# Gera o objetivo toolbox responsável por registrar as configurações do framewrok
toolbox = base.Toolbox()

# Cria o tipo de função fitness e indivíduo
creator.create("Maximization", base.Fitness, weights=(1.0,)) #-1 for min
creator.create("Cromossomo", list,fitness=creator.Maximization)

# Registra os nomes e os tipos de individuo, fiteness e população
toolbox.register("atributo", random.randint, 0, 1)
toolbox.register("solucaoFinal", tools.initRepeat, creator.Cromossomo, toolbox.atributo, n=len(produtos) )
toolbox.register("Populacao", tools.initRepeat, list, toolbox.solucaoFinal)

# Registra os operadores. Deve-se manter os nomes evaluate, mate, mutate e select
toolbox.register("evaluate", fitness_function)
# toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mate", tools.cxTwoPoints)
toolbox.register("mutate", tools.mutFlipBit, indpb = 0.05)
# toolbox.register("select", tools.selRoulette)
toolbox.register("select", tools.selTournament, tournsize = 3)


In [12]:
def statistics(individual):
  return individual.fitness.values

In [13]:
# Gera uma população para teste
testePop = toolbox.Populacao(n=10)
testePop

[[0, 0, 0, 1, 1, 0, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 0, 0, 1, 0, 0],
 [1, 1, 1, 0, 1, 1, 0, 1, 0, 1],
 [0, 1, 0, 1, 1, 0, 1, 0, 1, 0],
 [1, 1, 1, 0, 1, 0, 1, 0, 0, 0],
 [0, 1, 1, 1, 1, 0, 0, 1, 1, 1],
 [0, 1, 0, 0, 1, 1, 1, 0, 1, 0],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
 [0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
 [1, 1, 1, 0, 0, 0, 1, 0, 1, 0]]

In [14]:
# Gera o Algoritmo Genético com a estrutura e operadores cadastrados
pop = toolbox.Populacao(n=20)
prob_cx = 0.8
prob_mt = 0.1
nger = 100
hof = tools.HallOfFame(10)

stat = tools.Statistics(statistics)
stat.register("Melhor Solucao", numpy.max)
stat.register("Media da Populacao", numpy.mean)

finalPop, log = algorithms.eaSimple(pop, toolbox, prob_cx, prob_mt, nger, stat, hof, verbose = True)

melhorSolucao = tools.selBest(finalPop, 1)
for i in melhorSolucao:
  print(f'Solucao: {i} - Fitness {i.fitness}')

gen	nevals	Melhor Solucao	Media da Populacao
0  	20    	179           	28.95             
1  	15    	216           	58                
2  	18    	247           	78.75             
3  	18    	251           	155.45            
4  	16    	251           	188.6             
5  	14    	251           	226.2             
6  	17    	251           	247.15            
7  	16    	251           	238.45            
8  	16    	251           	238.45            
9  	20    	251           	248.85            
10 	18    	251           	238.45            
11 	18    	251           	251               
12 	14    	251           	251               
13 	17    	251           	248.55            
14 	17    	251           	238.45            
15 	18    	251           	251               
16 	14    	251           	238.45            
17 	20    	251           	238.45            
18 	16    	251           	213.35            
19 	17    	251           	221.3             
20 	20    	251           	247.65            
21 	16    

/home/a10/anaconda3/lib/python3.11/site-packages/deap/tools/crossover.py:67: FutureWarning: tools.cxTwoPoints has been renamed. Use cxTwoPoint instead.
  warnings.warn("tools.cxTwoPoints has been renamed. Use cxTwoPoint instead.",


In [15]:
for i in hof:
  print(f'Solucao: {i} - Fit: {i.fitness}')

Solucao: [1, 1, 0, 1, 0, 0, 1, 0, 0, 0] - Fit: (284.0,)
Solucao: [1, 1, 0, 0, 0, 1, 1, 0, 0, 0] - Fit: (259.0,)
Solucao: [1, 0, 1, 0, 0, 1, 1, 0, 0, 0] - Fit: (251.0,)
Solucao: [1, 0, 0, 1, 0, 0, 0, 0, 1, 0] - Fit: (247.0,)
Solucao: [1, 0, 1, 0, 1, 1, 0, 0, 0, 0] - Fit: (244.0,)
Solucao: [1, 1, 0, 1, 0, 0, 0, 0, 0, 0] - Fit: (217.0,)
Solucao: [1, 1, 0, 0, 0, 0, 1, 0, 0, 0] - Fit: (216.0,)
Solucao: [0, 1, 1, 0, 0, 1, 1, 0, 0, 0] - Fit: (216.0,)
Solucao: [1, 0, 1, 0, 0, 0, 1, 0, 0, 0] - Fit: (208.0,)
Solucao: [1, 0, 0, 0, 0, 1, 1, 0, 0, 0] - Fit: (202.0,)
